# Using models with templates

The advantage of a framework is that all its parts can interact with each other easily. Do you remember that Django is based on the Model-View-Template architecture? It consists of the model, which defines the data structure, the view, which handles the logic and interaction with the user, and the template, which controls the presentation and rendering of the data. In addition, we have forms to collect information from users. Let's see it in action — process and pass data from the database to our HTML templates and get it from HTML forms to save to the database.



### Initial setup
Let's create our music service. This service lets you listen to various tracks, search for specific songs, and mark your favorite ones.

First, let's create our project using the following command:

```django-admin startproject music_player```

Now we'll create an application inside the project directory. Let's call it play.

```python manage.py startapp play```

We've created a new application and need to add it to the INSTALLED_APPS list in the settings.py file.

```
INSTALLED_APPS = [
    ...	
    'play',
]
```

We don't need much for the prototype — let's use Django's User model and define another one ourselves in the models.py file:



In [ ]:
from django.contrib.auth.models import User
from django.db import models


class Song(models.Model):
    title = models.CharField(max_length=128)
    artist = models.CharField(max_length=128)
    path_to_file = models.FileField(upload_to='static/')
    favorite_by = models.ManyToManyField(User, related_name='favorite_songs')

All attributes of models are accessed through the dot. For example, if we create the song variable as an instance of the Song class, its attributes will be available using expressions like song.title, song.artist, etc.

The path_to_file is a relative path to a file from the upload_to directory. Don't redefine the default value for static files in the settings.py module, and use the static as a file source prefix. If we create an instance of the Song model where the value of the path_to_file field is text.txt, Django will look for the text.txt file in the static directory.

The next step is to place the music files in the static directory of our app.

Don't forget to migrate a newly created model:

In [ ]:
python manage.py makemigrations
python manage.py migrate

The next step is creating a template directory inside the application — play/templates/play. We'll need two pages for our educational purposes: song_list.html and play_page.html.

The idea is to turn it into dynamic content generated or customized based on user requests or inputs, using server-side technologies to fetch data from a database or perform calculations before delivering the content to the user. To achieve this, we can pass a context dictionary to the template to fill it with data from our models.

A context is a dictionary with variable names as the key and their values as the value. Assume that you defined the context variable song, assigned it the value of an instance of the Song class, and want to add it to your context dictionary. Here is how to do that:

In [ ]:
my_favorite_song = Song(title='All you need is a computer, and a little belief in yourself',
                        artist='Peachcake', 
                        path_to_file='file_aync_peachcake_2005.mp3')

context = {'song': my_favorite_song}

In this case, our context dictionary is {'song': my_favorite_song}. In models, all fields will be available for the template layout; you can even access foreign keys and their fields. You can pass context dictionaries to your views to make them available for your templates.

Fill the database with your favorite music tracks through the Django admin panel: create the superuser and access the panel at http://127.0.0.1:8000/admin/. To make our service work, let's create two significant components: views and URL paths. Since the primary purpose of our topic is to learn to use models with templates, we will only take a quick look at these steps.

# Views
In Django, web pages and other content (like JSON, spreadsheets, or PDFs) are delivered by views. Each view generally serves a specific function and has a particular template of HTML. For example, in our playback application, we'll need at least two view classes, one for the song list and one for the pages with a music player where we can listen to our songs.

Let's add a list of songs to the views.py file:




In [ ]:
from django.views import generic
from .models import Song

class IndexView(generic.ListView):
    template_name = 'play/song_list.html'
    context_object_name = 'songs'

    def get_queryset(self):
        return Song.objects.all()

As you can see, we're forming a QuerySet of song instances from the database to pass them to the template.

The second page is a detailed view of the exact music track (play/play_page.html). The view class for it may be like this:

In [ ]:
class MusicFileView(generic.DetailView):
    template_name = 'play/play_page.html'
    model = Song